In [ ]:
import scanpy as sc
import anndata 
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import os
import pandas as pd

In [ ]:
type="Aregs"

X = io.mmread(f"/home/ssloveff/lucas/loom/Seurat_dataset/Aregs/counts.mtx")

# create anndata object
adata = anndata.AnnData(
    X=X.transpose().tocsr()
)

cell_meta = pd.read_csv(f"/home/ssloveff/lucas/loom/Seurat_dataset/Aregs/metadata.csv")

with open(f"/home/ssloveff/lucas/loom/Seurat_dataset/Aregs/gene_names.csv", 'r') as f:
    gene_names = f.read().splitlines()

adata.obs = cell_meta
adata.obs.index = adata.obs['barcode']

adata.var.index = gene_names

pca = pd.read_csv(f"/home/ssloveff/lucas/loom/Seurat_dataset/Aregs/pca.csv")
pca.index = adata.obs.index

adata.obsm['X_pca'] = pca.to_numpy()
adata.obsm['X_umap'] = np.vstack((adata.obs['UMAP_1'].to_numpy(), adata.obs['UMAP_2'].to_numpy())).T

sc.pl.umap(adata, color='seurat_clusters', frameon=False, save=True)

In [ ]:
import scvelo as scv
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad

import matplotlib.pyplot as plt

scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo', facecolor='white', dpi=100, 
                               fontsize=6, color_map='viridis',
                               frameon=False)

In [ ]:
sample_loom_1="/home/ssloveff/lucas/loom/Young_TN_count.loom"
ldata1 = scv.read( sample_loom_1, cache=True)
#ldata1.obs['batch'] = '1'
#ldata1.obs['BATCH'] = '1'
ldata1

sample_loom_2="/home/ssloveff/lucas/loom/Young_3D_count.loom"
ldata2 = scv.read( sample_loom_2, cache=True)
#ldata2.obs['batch'] = '2'
#ldata2.obs['BATCH'] = '2'
ldata2

sample_loom_3="/home/ssloveff/lucas/loom/Young_14D_count.loom"
ldata3 = scv.read( sample_loom_3, cache=True)
#ldata3.obs['batch'] = '3'
#ldata3.obs['BATCH'] = '3'
ldata3

sample_loom_4="/home/ssloveff/lucas/loom/Aged_TN_count.loom"
ldata4 = scv.read( sample_loom_4, cache=True)
#ldata4.obs['batch'] = '4'
#ldata4.obs['BATCH'] = '4'
ldata4

sample_loom_5="/home/ssloveff/lucas/loom/Aged_3D_count.loom"
ldata5 = scv.read( sample_loom_5, cache=True)
#ldata5.obs['batch'] = '5'
#ldata5.obs['BATCH'] = '5'
ldata5

sample_loom_6="/home/ssloveff/lucas/loom/Aged_14D_count.loom"
ldata6 = scv.read( sample_loom_6, cache=True)
#ldata6.obs['batch'] = '6'
#ldata6.obs['BATCH'] = '6'
ldata6

# ldata1
barcodes = [bc.split(':')[1] for bc in ldata1.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata1.obs.index = barcodes
ldata1.obs.index[0:5]

# ldata2
barcodes = [bc.split(':')[1] for bc in ldata2.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata2.obs.index = barcodes
ldata2.obs.index[0:5]

# ldata3
barcodes = [bc.split(':')[1] for bc in ldata3.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata3.obs.index = barcodes
ldata3.obs.index[0:5]

# ldata4
barcodes = [bc.split(':')[1] for bc in ldata4.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata4.obs.index = barcodes
ldata4.obs.index[0:5]

# ldata5
barcodes = [bc.split(':')[1] for bc in ldata5.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata5.obs.index = barcodes
ldata5.obs.index[0:5]

# ldata6
barcodes = [bc.split(':')[1] for bc in ldata6.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata6.obs.index = barcodes
ldata6.obs.index[0:5]

ldata1.var.head()
ldata2.var.head()
ldata3.var.head()
ldata4.var.head()
ldata5.var.head()
ldata6.var.head()

ldata1.var_names_make_unique()
ldata2.var_names_make_unique()
ldata3.var_names_make_unique()
ldata4.var_names_make_unique()
ldata5.var_names_make_unique()
ldata6.var_names_make_unique()

ldata = sc.AnnData.concatenate(ldata1,ldata2,ldata3,ldata4,ldata5,ldata6,batch_key = 'BATCH')
ldata.obs


In [ ]:
adata = scv.utils.merge(adata, ldata)
adata.obs

scv.pp.filter_and_normalize(adata)

scv.pp.moments(adata, n_neighbors=30, n_pcs=30)

In [ ]:
import gc
gc.collect()

In [ ]:
temp_pre= f"Aregs_nue.in_process2"
if False==os.path.exists(f"{temp_pre}.velo.gz.h5ad"):
    scv.tl.recover_dynamics(adata, var_names='all', n_jobs=40)
    scv.tl.velocity(adata, mode='dynamical')
    adata.write(f"{temp_pre}.velo.gz.h5ad", compression='gzip')
    print(">>Write to file")
else:
    adata = sc.read(f"{temp_pre}.velo.gz.h5ad", compression='gzip', ext="h5ad")
    print(">>read from file")

scv.tl.velocity_graph(adata, n_jobs=40)

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap")

In [ ]:
scv.pl.velocity_embedding(adata, basis="umap", save='figures/Aregs/embedding_Aregs.pdf')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap', color=['seurat_clusters'], figsize=(8,8), 
    palette =  ("#f26f66","#4962a1","#63b9bd",'#2F4858',"#8572ad",'#BBA6E5','#539075',"#3082b4","#ccaf74"), 
    arrow_size=2, linewidth=1.5, legend_fontsize=25, dpi=900, save='figures/Aregs/embedding_stream_seurat_clusters_Aregs.svg', title='')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap', color=['celltype'], figsize=(8,8), 
    palette =  ('#fdd171','#CCC9E6','#fac1b8'), 
    arrow_size=2, linewidth=1.2, legend_fontsize=25, dpi=900, density = 1.2, save='figures/Aregs/embedding_stream_celltype_Aregs.svg', title='')

In [ ]:
scv.pl.velocity_embedding_grid(adata, basis='umap', color='celltype', figsize=(8,8), 
    arrow_size=2.2,  arrow_length=2, size=120, alpha=0.3, save='figures/Aregs/embedding_grid_Aregs.pdf', title='', scale=0.25)

In [ ]:
scv.pl.velocity_embedding(adata, arrow_length=2, arrow_size=1.5, dpi=900, figsize=(8,8), save='figures/Aregs/embedding_stream2_Aregs.pdf')

In [ ]:
scv.pl.velocity(adata,
['F3','Igf1','Igfbp3','Gas6','Timp1',"Penk"],
figsize=(5,5), dpi=300, save='figures/Aregs/Interprete velocity_Aregs.png', ncols=2)

In [ ]:
scv.pl.velocity(adata,
["Gas6",'Axl',"Trem2","Mgp","Fmo2","Vit","Agt","Grem2"],
figsize=(5,5), dpi=300, save='figures/Aregs/Interprete velocity_Aregs-2.png', ncols=2)